[Link na zadatke](http://www.fer.unizg.hr/_download/repository/vjezba_3%5B4%5D.pdf)

In [1]:
import numpy as np
import pandas as pd
import sympy as sp
import math
from common import *
from sympy.abc import x, y
from sympy import Matrix

norm = np.linalg.norm
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

In [2]:
%%capture
%run Lab02.ipynb

In [3]:
f1 = 100 * (y - x ** 2)  ** 2 + (1 - x) ** 2
f1_x0 = pt(-1.9, 2.0)
f1_xmin = pt(1, 1)
f1_min = 0

f2 = (x - 4) ** 2 + 4 * (y - 2) ** 2
f2_x0 = pt(0.1, 0.3)
f2_xmin = pt(4, 2)
f2_min = 0

f3 = (x - 2) ** 2 + (y + 3) ** 2
f3_x0 = pt(0, 0)
f3_xmin = pt(2, -3)
f3_min = 0

f4 = (x - 3) ** 2 + y ** 2
f4_x0 = pt(0, 0)
f4_xmin = pt(3, 0)
f4_min = 0

In [4]:
@counted
def evalf(f, pt):
    return sp.lambdify([(x,y)], f)(pt)

@counted
def gradient(f):
    jacobian = Matrix([f]).jacobian((x, y))
    return lambda pt: sp.lambdify([(x, y)], jacobian, 'numpy')(pt).reshape(-1)

@counted
def hessian(f, constraints = []):
    return sp.lambdify([(x, y)], sp.hessian(f, [x, y], constraints))

In [5]:
def reset():
    hessian.called = 0
    gradient.called = 0
    evalf.called = 0

In [6]:
gradient(f1)((1,0))

array([ 400, -200])

### Gradient Descent

[Sympy gradient and Hessian](https://stackoverflow.com/questions/39558515/how-to-get-the-gradient-and-hessian-sympy)

Postupak treba podržavati
dva načina rada: u prvom načinu postupak se uvijek pomiče u novu točku za čitav iznos dobivenog pomaka
(nenormirani vektor gradijenta), dok u drugom načinu postupak korištenjem metode zlatnog reza pronalazi
optimalan iznos pomaka na pravcu. Postupak je potrebno zaustaviti kada euklidska norma gradijenta postane
manja od neke zadane preciznosti

In [7]:
def gradient_descent(f, x0, e = 1e-6, max_iter = 1000,
                     optimize_move_callable = lambda *x: 1.0):
    gradient_callable = gradient(f)
    gradient_vector = gradient_callable(x0)
    x = np.copy(x0)
    i = 0
    
    while np.linalg.norm(gradient_vector) > e and i < max_iter:
        vector_mul = optimize_move_callable(
            lambda d: evalf(f, x - d * gradient_vector), 0.1, e)
        x -= vector_mul * gradient_vector
        gradient_vector = gradient_callable(x)
        i += 1
        
    return x

reset()
gradient_descent(f1, f1_x0)

/usr/lib/python3.6/site-packages/numpy/__init__.py:1: RuntimeWarning: overflow encountered in double_scalars
  """
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in subtract
  # This is added back by InteractiveShellApp.init_path()


array([ nan,  inf])

In [8]:
def optimize_move_callable(f, d, epsilon):
    l, r = unimodal(f, d)
    return golden_section(f, l, r, epsilon)

gradient_descent(f1, f1_x0, max_iter=10, optimize_move_callable=optimize_move_callable)

array([ 1.20875647,  1.46153288])

### Newton-Rhapson
Postupak treba podržavati dva načina rada: u prvom načinu postupak se uvijek pomiče u novu točku za čitav
iznos dobivenog pomaka, dok u drugom načinu postupak korištenjem metode zlatnog reza pronalazi
optimalan iznos pomaka. Postupak je potrebno zaustaviti kada euklidska norma pomaka postane manja od
neke zadane preciznosti

In [9]:
def newton_rhapson(f, x0, e = 1e-6, optimize_move_callable=lambda *x: 1.0, max_iter=1000):
    x = np.copy(x0)

    gradient_callable = gradient(f)
    hessian_callable = hessian(f)
    gradient_vector = gradient_callable(x)
    
    i = 0
    while norm(gradient_vector) > e and i < max_iter:
        i += 1
        
        gradient_vector = np.dot(np.linalg.inv(hessian_callable(x)), 
                                 gradient_callable(x)).reshape(-1)
        x -= gradient_vector * optimize_move_callable(
            lambda d: evalf(f, x - d * gradient_vector), 0.1, e)
        
    return x

In [10]:
newton_rhapson(f1, f1_x0)

array([ 1.,  1.])

In [11]:
f1_xmin

array([ 1.,  1.])

### [Box](http://www.fer.unizg.hr/_download/repository/box.html)

In [45]:
def box(f, x0, g, xd, xg, alpha=1.3, epsilon1 = 1e-6, epsilon2 = 1e-6, max_iter=1000):
    #assert all(xd <= x0 <= xg)
    #assert g(x0) >= 0
    
    xc = np.copy(x0)
    n = len(x0)
    h, h2 = -1, -2
    
    reflect = lambda xc, xh: xc + alpha * (xc - xh)
    x = np.ones([2 * n, n])
    
    for t in range(2 * n):
        for i in range(n):
            R = np.random.randn()
            x[t][i] = xd[i] + R * (xg[i] - xd[i])
        while any(g(x[t]) < 0):
            x[t] = 0.5 * (x[t] + xc)
            xc = np.mean(x, axis=0)
            
    i = 0
    while i < max_iter:
        i += 1
        x = sorted(x, key=lambda x: evalf(f, x))
        
        xc = np.mean(x[:h], axis=0)
        xr = reflect(xc, x[h])
        
        for i in range(n):
            if xr[i] < xd[i]:
                xr[i] = xd[i]
            elif xr[i] > xg[i]:
                xr[i] = xg[i]
                
        while np.any(g(xr) < 0):
            xr = 0.5 * (xr + xc)
        if evalf(f, xr) > evalf(f, x[h2]):
            xr = 0.5 * (xr + xc)
        x[h] = xr
        
        if norm(x[h] - xc) <= epsilon1 or abs(evalf(f, x[h]) - evalf(f, xc)) < epsilon2:
            break
            
    return xc

box(f2, pt(50, 50), lambda x: x, pt(0,0), pt(10,10))       

array([ 3.9989882 ,  2.00082354])

# 1.
Primijenite postupak gradijentnog spusta na funkciju 3, uz i bez određivanja optimalnog iznosa koraka. Što možete zaključiti iz rezultata?

In [12]:
gradient_descent(f3, f3_x0)

array([ 0.,  0.])

In [13]:
gradient_descent(f3, f3_x0, optimize_move_callable=optimize_move_callable)

array([ 2.00000004, -3.00000006])

# 2.
Primijenite postupak gradijentnog spusta i Newton-Raphsonov postupak na funkcije 1 i 2 s određivanjem optimalnog iznosa koraka. Kako se Newton-Raphsonov postupak ponaša na ovim funkcijama? Ispišite broj izračuna funkcije, gradijenta i Hesseove matrice.

In [14]:
reset()
gradient_descent(f1, f1_x0, optimize_move_callable=optimize_move_callable)

array([ 1.1295641 ,  1.27619717])

In [15]:
reset()
gradient_descent(f2, f2_x0, optimize_move_callable=optimize_move_callable)

array([ 3.99999965,  2.00000005])

In [16]:
reset()
newton_rhapson(f1, f1_x0, optimize_move_callable=optimize_move_callable)

array([ 1.,  1.])

In [17]:
reset()
newton_rhapson(f2, f2_x0, optimize_move_callable=optimize_move_callable)

array([ 4.,  2.])

# 3. 
Primijenite postupak po Boxu na funkcije 1 i 2 uz implicitna ograničenja: (x2-x1 >= 0), (2-x1 >= 0) i
eksplicitna ograničenja prema kojima su sve varijable u intervalu [-100, 100]. Mijenja li se položaj
optimuma uz nametnuta ograničenja? 

In [46]:
box(f1, f1_x0, lambda x: np.array([x[1] - x[0], 2 - x[0]]), pt(-100, -100), pt(100, 100))

array([ 0.01018513,  0.01018568])

In [47]:
box(f2, f2_x0, lambda x: np.array([x[1] - x[0], 2 - x[0]]), pt(-100, -100), pt(100, 100))

array([ 1.99999798,  2.0013366 ])

# 4. 
Primijenite postupak transformacije u problem bez ograničenja na funkcije 1 i 2 s ograničenjima iz
prethodnog zadatka (zanemarite eksplicitna ograničenja). Novodobiveni problem optimizacije bez
ograničenja minimizirajte koristeći postupak Hooke-Jeeves ili postupak simpleksa po Nelderu i
Meadu. Može li se korištenjem ovog postupka pronaći optimalno rješenje problema s ograničenjima?
Ako ne, probajte odabrati početnu točku iz koje je moguće pronaći rješenje. 

# 5.
Za funkciju 4 s ograničenjima (3-x1-x2>=0), (3+1.5*x1-x2>=0) i (x2-1=0) probajte pronaći
minimum koristeći postupak transformacije u problem bez ograničenja (također koristite HookeJeeves
ili postupak simpleksa po Nelderu i Meadu za minimizaciju). Probajte kao početnu točku
postaviti neku točku koja ne zadovoljava ograničenja nejednakosti (primjerice točku (5,5)) te
pomoću postupka pronalaženja unutarnje točke odredite drugu točku koja zadovoljava ograničenja
nejednakosti te ju iskoristite kao početnu točku za postupak minimizacije. 